# Partial Wave Analysis of $\eta\pi$

In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

In [ ]:
using TypedTables
using DelimitedFiles
using Plots
using PartialWaveFunctions
using Cuba
using QuadGK
using Optim
using Test
using ForwardDiff

## Load data

In [ ]:
data = let
    M = readdlm(joinpath("data","data_metapi_costheta_phi.txt"))
    Table(mηπ=M[:,1], cosθ=M[:,2], ϕ=M[:,3])
end

## Build the model

The basis function are the orthonormal rotation functions
$$
\Psi_{LM}(\Omega) = \sqrt{\frac{2L+1}{2\pi}}d_{M0}^{L}(\theta)\,\sin(M\phi)
$$

In [ ]:
ΨLM(cosθ,ϕ,L,M) = sqrt((2L+1)/(4π))*wignerd(L,M,0,cosθ)*sin(M*ϕ)*sqrt(2)

One can check orthogonality of the functions

In [ ]:
let M=1, L=5
    4π*cuhre((x,f)->f[1] = abs2(ΨLM(2x[1]-1,π*(2*x[2]-1),L,M)),2,1).integral[1]
end

In [ ]:
const LMs = NamedTuple{(:L,:M)}.([(1,1),(2,1),(4,1)])

The model of the amplitude is a truncated series of the partial waves.

We truncate at $L<5$ and $M<2$. $L=0$ and $M=0$ are forbidden by the selection rule (*)

In [ ]:
model(cosθ,ϕ; pars) = sum(c*ΨLM(cosθ,ϕ,L,M) for (c,(L,M)) in zip(pars, LMs))

In [ ]:
@test model(0.1,0.1; pars=rand(length(LMs)) + 1im*rand(length(LMs))) != 0.0

Plot the model in $\cos\theta \times \phi$ coordinates

In [ ]:
let pars = rand(length(LMs)) + 1im*rand(length(LMs))
    cosθv = range(-1,1,length=100)
    ϕv = range(-π,π,length=100)
    calv = [model(cosθ,ϕ; pars=pars) for ϕ in ϕv, cosθ in cosθv]
    heatmap(cosθv, ϕv, abs2.(calv), lab="",
        color=cgrad(:viridis, scale=:exp))
end

In [ ]:
ellh(;data, pars) = -sum(log, abs2(model(cosθ,ϕ; pars=pars)) for (_,cosθ,ϕ) in data) + sum(abs2,pars)

In [ ]:
@time ellh(;data=data, pars=rand(length(LMs)) + 1im*rand(length(LMs)))

In [ ]:
let mηπ_lims = (1.5,1.54)
    ldata = filter(x->1.5<x.mηπ<1.54, data)
    Np = length(LMs)
    fold(x) = x[1:Np] + 1im .* x[(Np+1):2Np]
    init_pars = fold(2rand(2Np).-1) # get random starting parameters
    init_pars .*= sqrt(length(data)/sum(abs2,init_pars)) # normalize
    #
    f(x) = ellh(;data=ldata, pars=x)
    f′(x) = fold(ForwardDiff.gradient(p->f(fold(p)), vcat(real(x), imag(x))))
    f′!(stor,x) = copyto!(stor,f′(x))
    #
    Optim.optimize(f, f′!, init_pars, BFGS(),
                Optim.Options(show_trace = true))
end

Check if the constraint is fulfilled

In [ ]:
sum(abs2,Optim.minimizer(ans))

## Mass production:
 - create a settings file several fit attempts from random starting points

In [ ]:
function fit_data!(settings)
    ldata = settings["data"]
    _Natt = settings["Natt"]
    #
    Np = length(LMs)
    fold(x) = x[1:Np]+x[Np+1:2Np]
    #
    f(x) = ellh(;data=ldata, pars=x)
    f′(x) = fold(ForwardDiff.gradient(p->f(fold(p)), vcat(real(x), imag(x))))
    f′!(stor,x) = copyto!(stor,f′(x))
    # 
    frs = [let
        init_pars = fold(2rand(2Np).-1) # get random starting parameters
        init_pars .*= sqrt(length(data)/sum(abs2,init_pars)) # normalize
        Optim.optimize(f, f′!, init_pars, BFGS(),
                    Optim.Options(show_trace = settings["show_trace"]))
    end for e in 1:_Natt]
    settings["fit_results"] = frs
end

In [ ]:
settings = Dict(
    "data"=>filter(x->1.5<x.mηπ<1.54, data),
    "Natt"=>100,
    "show_trace"=>false)
fit_data!(settings);

## Analysis of the fit results

In [ ]:
Optim.minimizer(settings["fit_results"][1])

In [ ]:
tfr = Table(
    [(st = Optim.converged(fr), min = Optim.minimum(fr), pars = Optim.minimizer(fr))
        for fr in settings["fit_results"]])

In [ ]:
converged = filter(x->x.st, tfr);

In [ ]:
histogram(converged.min, bins=100)

In [ ]:
bestpars = converged[findmin(converged.min)[2]].pars

In [ ]:
let pars = bestpars
    cosθv = range(-1,1,length=100)
    ϕv = range(-π,π,length=100)
    calv = [model(cosθ,ϕ; pars=pars) for ϕ in ϕv, cosθ in cosθv]
    heatmap(cosθv, ϕv, abs2.(calv), lab="",
        color=cgrad(:viridis, scale=:exp))
end

In [ ]:
histogram2d(settings["data"].cosθ, settings["data"].ϕ, bins=50)

In [ ]:
let
    Nd = length(settings["data"])
    Nb = 50
    stephist(settings["data"].cosθ, bins=Nb, lc=:black, lab="data")
    Int(cosθ) = quadgk(ϕ->abs2(model(cosθ,ϕ; pars=bestpars)),-π,π)[1] / Nb * (1+1)
    plot!(Int, -1, 1, lw=3, lab="model")
end